In [1]:
# from git
# !pip install git+https://github.com/Bots-Avatar/ExplainitAll -q

# from zip
!pip install /am/ExplainitAll.zip -q

In [1]:
import os
# os.environ['TEST_MODE_ON_LOW_SPEC_PC'] = 'True'

In [2]:
from explainitall.QA.interp_qa.KNNWithGenerative import FredStruct, PromptBot
from explainitall.QA.extractive_qa_sbert.SVDBert import SVDBertModel
from explainitall.QA.extractive_qa_sbert.QABotsBase import cos_dist
from sklearn.neighbors import KNeighborsClassifier
from sentence_transformers import SentenceTransformer
from explainitall.gui.interface import set_verbosity_error
set_verbosity_error()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
path = 'FractalGPT/SbertSVDDistil'
sbert = SentenceTransformer(path)
sbert[0].auto_model = SVDBertModel.from_pretrained(path)

In [4]:

if os.getenv('TEST_MODE_ON_LOW_SPEC_PC') == 'True':
    fred = FredStruct('t5-small' )
else:
    fred =  FredStruct('SiberiaSoft/SiberianFredT5-instructor')
    

tokenizer_config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.76M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.74k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/217 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/848 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.48G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [5]:
datasetQ = [
    'Привет!',
    'У меня нет имени, я простая вопросно-ответная система',
    'Мне 3 года',
    'Машина это средство передвижения',
    'Полупроводнико́вый трио́д — электронный компонент из полупроводникового материала, способный небольшим входным сигналом управлять значительным током в выходной цепи, что позволяет использовать его для усиления, генерирования, коммутации и преобразования электрических сигналов.',
    'В классическом понимании поезд представляет собой движущуюся по пути цепь из соединённых между собой «повозок» — единиц подвижного состава.',
    'С точки зрения научной систематики, домашняя кошка — млекопитающее семейства кошачьих отряда хищных. Одни исследователи рассматривают домашнюю кошку как подвид дикой кошки[5], другие — как отдельный биологический вид[6].',
]

In [6]:
# Только ответы
vects_x = sbert.encode(datasetQ)
m = vects_x.mean(axis=0)
s = vects_x.std(axis=0)
knn_vects_x = (vects_x - m) / s
knn = KNeighborsClassifier(metric=cos_dist)
knn.fit(knn_vects_x, datasetQ)

KNeighborsClassifier(metric=<function cos_dist at 0x7fe2f695ad40>)

In [7]:
bot = PromptBot(knn, sbert, fred, datasetQ, device='cpu')

In [8]:
bot.get_answers('Насколько ты старый?', top_k=4)

'Мне 3 года.'

In [9]:
bot.get_answers('Расскажи про машины', top_k=2)

'Привет! Машины - это транспортные средства, которые используются для перемещения людей и грузов по дорогам. Они могут быть как с двигателем внутреннего сгорания, так и с электрическим двигателем. В зависимости от типа двигателя машины могут быть грузовыми, пассажирскими или сельскохозяйственными..'